# Datathon FME 2022

In [364]:
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier

In [365]:
df_cities = pd.read_csv(r"C:\Users\Álex\Documents\Datathon\data\datathonfme2022-main\datathonfme2022-main\accenture_challenge\datathon_SC_ACN_22\cities_data.csv", sep=";", encoding="UTF-8")
df_orders = pd.read_csv(r"C:\Users\Álex\Documents\Datathon\data\datathonfme2022-main\datathonfme2022-main\accenture_challenge\datathon_SC_ACN_22\orders.csv", sep=";", encoding="UTF-8")

In [366]:
df_cities.head()

,city_from_name,city_to_name,city_from_coord,city_to_coord,distance
0,Dusseldorf,Zaragoza,"(51.2254018, 6.7763137)","(41.6521342, -0.8809428)",1214.4772
1,Dusseldorf,Venlo,"(51.2254018, 6.7763137)","(51.39244885, 6.1511724144122955)",47.3876
2,Dusseldorf,Bratislava,"(51.2254018, 6.7763137)","(48.1516988, 17.1093063)",819.2894
3,Dusseldorf,Hamburg,"(51.2254018, 6.7763137)","(53.550341, 10.000654)",339.2215
4,Dusseldorf,Rome,"(51.2254018, 6.7763137)","(41.8933203, 12.4829321)",1125.0290


In [367]:
df_orders.head()

,order_id,origin_port,3pl,customs_procedures,logistic_hub,customer,product_id,units,late_order
0,366c7a3d298f,Rotterdam,v_002,DTP,Venlo,Marseille,1692723,583,True
1,45f906331e10,Rotterdam,v_004,CRF,Rome,Marseille,1644308,459,False
2,ac84a6e4af0f,Athens,v_002,CRF,Venlo,Paris,1684170,464,True
3,f5e98cb29790,Rotterdam,v_004,CRF,Lille,Milan,1620510,678,False
4,a9e7c9bee35b,Barcelona,v_002,CRF,Venlo,Berlin,1699372,353,False


## Feature Engineering

In this step we added some interesting features to the dataset such as the distance or the type of material being handled

In [369]:
distances = []
for i in df_orders.values:
    dist = ((df_cities[(df_cities["city_from_name"] == i[1]) & (df_cities["city_to_name"] == i[4])]["distance"].values) or (df_cities[(df_cities["city_from_name"] == i[4]) & (df_cities["city_to_name"] == i[1])]["distance"].values))
    if (dist):
        distances.append(dist[0])
    else:
        distances.append(0)

C:\Users\Álex\AppData\Local\Temp\ipykernel_18188\886607660.py:3: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  dist = ((df_cities[(df_cities["city_from_name"] == i[1]) & (df_cities["city_to_name"] == i[4])]["distance"].values) or (df_cities[(df_cities["city_from_name"] == i[4]) & (df_cities["city_to_name"] == i[1])]["distance"].values))
C:\Users\Álex\AppData\Local\Temp\ipykernel_18188\886607660.py:4: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if (dist):


In [370]:
df_orders["dist_to_hub"] = distances

In [371]:
df_orders.values[0]

array(['366c7a3d298f', 'Rotterdam', 'v_002', 'DTP', 'Venlo', 'Marseille',
       1692723, 583, True, 130.0459], dtype=object)

In [372]:
distances = []
for i in df_orders.values:
    dist = ((df_cities[(df_cities["city_from_name"] == i[4]) & (df_cities["city_to_name"] == i[5])]["distance"].values) or (df_cities[(df_cities["city_from_name"] == i[5]) & (df_cities["city_to_name"] == i[4])]["distance"].values))
    if (dist):
        distances.append(dist[0])
    else:
        distances.append(0)

C:\Users\Álex\AppData\Local\Temp\ipykernel_18188\3060743590.py:3: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  dist = ((df_cities[(df_cities["city_from_name"] == i[4]) & (df_cities["city_to_name"] == i[5])]["distance"].values) or (df_cities[(df_cities["city_from_name"] == i[5]) & (df_cities["city_to_name"] == i[4])]["distance"].values))
C:\Users\Álex\AppData\Local\Temp\ipykernel_18188\3060743590.py:4: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if (dist):


In [373]:
df_orders["hub_to_custom"] = distances

In [374]:
df_orders.head()

,order_id,origin_port,3pl,customs_procedures,logistic_hub,customer,product_id,units,late_order,dist_to_hub,hub_to_custom
0,366c7a3d298f,Rotterdam,v_002,DTP,Venlo,Marseille,1692723,583,True,130.0459,902.0420
1,45f906331e10,Rotterdam,v_004,CRF,Rome,Marseille,1644308,459,False,1269.2365,604.0216
2,ac84a6e4af0f,Athens,v_002,CRF,Venlo,Paris,1684170,464,True,2029.5726,392.9249
3,f5e98cb29790,Rotterdam,v_004,CRF,Lille,Milan,1620510,678,False,173.9644,733.8784
4,a9e7c9bee35b,Barcelona,v_002,CRF,Venlo,Berlin,1699372,353,False,1153.4178,512.8188


In [375]:
df_orders.to_csv(r"..\data\orders.csv")

### Mixing Features

In [376]:
from sklearn.linear_model import LinearRegression
df_onehot = pd.get_dummies(df_orders, prefix = ["customs_procedures", "3pl"], columns = ["customs_procedures", "3pl"])

df_onehot.head()

,order_id,origin_port,logistic_hub,customer,product_id,units,late_order,dist_to_hub,hub_to_custom,customs_procedures_CRF,customs_procedures_DTD,customs_procedures_DTP,3pl_v_001,3pl_v_002,3pl_v_003,3pl_v_004
0,366c7a3d298f,Rotterdam,Venlo,Marseille,1692723,583,True,130.0459,902.0420,0,0,1,0,1,0,0
1,45f906331e10,Rotterdam,Rome,Marseille,1644308,459,False,1269.2365,604.0216,1,0,0,0,0,0,1
2,ac84a6e4af0f,Athens,Venlo,Paris,1684170,464,True,2029.5726,392.9249,1,0,0,0,1,0,0
3,f5e98cb29790,Rotterdam,Lille,Milan,1620510,678,False,173.9644,733.8784,1,0,0,0,0,0,1
4,a9e7c9bee35b,Barcelona,Venlo,Berlin,1699372,353,False,1153.4178,512.8188,1,0,0,0,1,0,0


In [377]:
from sklearn.feature_selection import mutual_info_regression
df_onehot.drop(["order_id", "origin_port", "logistic_hub", "customer", "product_id"], inplace=True, axis=1)
X = df_onehot.copy()
y = X.pop("late_order")
def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores
discrete_features = X.dtypes == int
mi_scores = make_mi_scores(X, y, discrete_features)
mi_scores[::3]

dist_to_hub               0.030304
customs_procedures_DTP    0.006156
3pl_v_001                 0.002445
3pl_v_003                 0.000000
Name: MI Scores, dtype: float64

In [378]:
df_onehot["latecount_v001"] = df_onehot.groupby("3pl_v_001")["late_order"].transform("count") * df_onehot["3pl_v_001"]
df_onehot["latecount_v002"] = df_onehot.groupby("3pl_v_002")["late_order"].transform("count") * df_onehot["3pl_v_002"]
df_onehot["latecount_v003"] = df_onehot.groupby("3pl_v_003")["late_order"].transform("count") * df_onehot["3pl_v_003"]
df_onehot["latecount_v004"] = df_onehot.groupby("3pl_v_004")["late_order"].transform("count") * df_onehot["3pl_v_004"]

In [379]:
df_onehot["3pl counts"] = df_onehot["latecount_v001"] + df_onehot["latecount_v002"] + df_onehot["latecount_v003"] + df_onehot["latecount_v004"]

In [380]:
df_onehot = df_onehot.drop(["latecount_v001", "latecount_v002", "latecount_v003", "latecount_v004"], axis=1)
#df_onehot = df_onehot.drop(["3pl_v_001", "3pl_v_002", "3pl_v_003", "3pl_v_004"], axis = 1)

In [381]:
df_onehot.head()

,units,late_order,dist_to_hub,hub_to_custom,customs_procedures_CRF,customs_procedures_DTD,customs_procedures_DTP,3pl_v_001,3pl_v_002,3pl_v_003,3pl_v_004,3pl counts
0,583,True,130.0459,902.0420,0,0,1,0,1,0,0,54937
1,459,False,1269.2365,604.0216,1,0,0,0,0,0,1,44429
2,464,True,2029.5726,392.9249,1,0,0,0,1,0,0,54937
3,678,False,173.9644,733.8784,1,0,0,0,0,0,1,44429
4,353,False,1153.4178,512.8188,1,0,0,0,1,0,0,54937


## Plottings

In [382]:
import matplotlib.pyplot as plt

In [383]:
X = df_onehot.copy()
y = X.pop("late_order")

## Machine Learning

In [384]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=53)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [385]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy')
classifier.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=10)

In [386]:
y_pred = classifier.predict(X_test) 
y_pred

array([ True, False, False, ..., False,  True, False])

In [393]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
from sklearn.metrics import accuracy_score
print ("Accuracy : ", accuracy_score(y_test, y_pred))
cm

Accuracy :  0.7274238711935597


array([[15231,  2207],
       [ 4023,  1395]], dtype=int64)

In [388]:
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor(max_leaf_nodes = 100, random_state=1)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

In [390]:
unscaled = sc.inverse_transform(X_test)
for i in range(len(predictions)):
    if predictions[i] < 0.2:
        print(unscaled[i])

[3.9200000e+02 1.1534178e+03 3.9292490e+02 1.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 6.9388939e-18 1.0000000e+00
 4.4429000e+04]
[5.0700000e+02 1.7730750e+02 6.6503760e+02 0.0000000e+00 1.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 6.9388939e-18 1.0000000e+00
 4.4429000e+04]
[4.9500000e+02 1.2121854e+03 1.2171632e+03 1.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 1.0000000e+00 6.9388939e-18 0.0000000e+00
 5.4937000e+04]
[2.810000e+02 1.300459e+02 6.524192e+02 0.000000e+00 1.000000e+00
 0.000000e+00 0.000000e+00 0.000000e+00 1.000000e+00 0.000000e+00
 6.933000e+03]
[4.8400000e+02 1.3004590e+02 1.1590472e+03 0.0000000e+00 1.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 1.0000000e+00 0.0000000e+00
 6.9330000e+03]
[4.3100000e+02 1.1356449e+03 8.1604970e+02 1.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 6.9388939e-18 1.0000000e+00
 4.4429000e+04]
[4.0600000e+02 1.3004590e+02 6.9634850e+02 0.0000000e+00 0.0000

[4.3600000e+02 1.7730750e+02 1.6251608e+03 0.0000000e+00 1.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 6.9388939e-18 1.0000000e+00
 4.4429000e+04]
[4.3000000e+02 1.7396440e+02 7.4499130e+02 0.0000000e+00 1.0000000e+00
 0.0000000e+00 0.0000000e+00 1.0000000e+00 6.9388939e-18 0.0000000e+00
 5.4937000e+04]
[3.3400000e+02 1.1356449e+03 8.1604970e+02 1.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 6.9388939e-18 1.0000000e+00
 4.4429000e+04]
[3.4100000e+02 2.0295726e+03 8.1568540e+02 1.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 1.0000000e+00 6.9388939e-18 0.0000000e+00
 5.4937000e+04]
[4.7500000e+02 1.3004590e+02 6.3941130e+02 0.0000000e+00 0.0000000e+00
 1.0000000e+00 0.0000000e+00 1.0000000e+00 6.9388939e-18 0.0000000e+00
 5.4937000e+04]
[3.7000000e+02 1.3004590e+02 5.3247690e+02 1.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 6.9388939e-18 1.0000000e+00
 4.4429000e+04]
[4.6000000e+02 4.1444890e+02 9.2823280e+02 1.0000000

[5.4800000e+02 4.1444890e+02 1.9580004e+03 1.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 1.0000000e+00 6.9388939e-18 0.0000000e+00
 5.4937000e+04]
[5.2100000e+02 1.1356449e+03 1.5346302e+03 0.0000000e+00 0.0000000e+00
 1.0000000e+00 0.0000000e+00 1.0000000e+00 6.9388939e-18 0.0000000e+00
 5.4937000e+04]
[5.5500000e+02 1.3004590e+02 6.9634850e+02 1.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 6.9388939e-18 1.0000000e+00
 4.4429000e+04]
[4.6700000e+02 1.7730750e+02 6.6503760e+02 1.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 1.0000000e+00 6.9388939e-18 0.0000000e+00
 5.4937000e+04]
[5.1400000e+02 1.1356449e+03 2.2978442e+03 0.0000000e+00 1.0000000e+00
 0.0000000e+00 0.0000000e+00 1.0000000e+00 6.9388939e-18 0.0000000e+00
 5.4937000e+04]
[5.5400000e+02 1.1494065e+03 2.4101700e+02 1.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 6.9388939e-18 1.0000000e+00
 4.4429000e+04]
[4.0400000e+02 1.1356449e+03 8.1604970e+02 1.0000000

[5.610000e+02 1.300459e+02 5.128188e+02 0.000000e+00 1.000000e+00
 0.000000e+00 0.000000e+00 0.000000e+00 1.000000e+00 0.000000e+00
 6.933000e+03]
[4.8200000e+02 4.1444890e+02 9.0123530e+02 0.0000000e+00 1.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 6.9388939e-18 1.0000000e+00
 4.4429000e+04]
[5.1900000e+02 1.3004590e+02 1.4369241e+03 0.0000000e+00 1.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 6.9388939e-18 1.0000000e+00
 4.4429000e+04]
[3.2400000e+02 2.0295726e+03 1.4369241e+03 1.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 1.0000000e+00 6.9388939e-18 0.0000000e+00
 5.4937000e+04]
[4.4900000e+02 1.1534178e+03 8.1568540e+02 0.0000000e+00 1.0000000e+00
 0.0000000e+00 0.0000000e+00 1.0000000e+00 6.9388939e-18 0.0000000e+00
 5.4937000e+04]
[4.7400000e+02 1.1534178e+03 5.1281880e+02 1.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 1.0000000e+00 6.9388939e-18 0.0000000e+00
 5.4937000e+04]
[4.720000e+02 1.739644e+02 7.338784e+02 1.000000e+00 0.000000e+

In [391]:
unscaled

array([[3.9200000e+02, 1.1534178e+03, 3.9292490e+02, ..., 6.9388939e-18,
        1.0000000e+00, 4.4429000e+04],
       [6.2900000e+02, 2.5702510e+02, 1.5085357e+03, ..., 6.9388939e-18,
        1.0000000e+00, 4.4429000e+04],
       [4.6400000e+02, 1.2121854e+03, 1.5397663e+03, ..., 6.9388939e-18,
        0.0000000e+00, 5.4937000e+04],
       ...,
       [5.1900000e+02, 1.5970905e+03, 1.1013447e+03, ..., 6.9388939e-18,
        0.0000000e+00, 5.4937000e+04],
       [4.8800000e+02, 1.7396440e+02, 8.4730820e+02, ..., 6.9388939e-18,
        1.0000000e+00, 4.4429000e+04],
       [4.3600000e+02, 1.7730750e+02, 9.8098220e+02, ..., 6.9388939e-18,
        1.0000000e+00, 4.4429000e+04]])

In [394]:
predictions

array([0.13814757, 0.44551282, 0.36335079, ..., 0.35105859, 0.31386861,
       0.12616372])